In [2]:
# Importing necessary libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Flatten, InputLayer
import keras


In [3]:
# Reading the data
train = pd.read_csv('train.csv')
trainX, trainY = train.iloc[:, :train.shape[1]-1], train.iloc[:, train.shape[1]-1]


In [4]:
# There are a total of 43 categorical columns
categoricals = trainX.loc[:, trainX.dtypes == 'O'].columns
len(categoricals) # 43


43

In [5]:
# Preprocessing step: 
# One Hot Encoder cannot work with NaN, hence filling NaN with mode of categorical columns
cat_features = trainX.loc[:, categoricals]
cat_features = cat_features.fillna(cat_features.mode().iloc[0, :])
# One hot encoding these features
ohe = OneHotEncoder(handle_unknown='ignore')
res = ohe.fit_transform(cat_features).toarray()
cols = np.array([])
for i in range(cat_features.shape[1]):
    cols = np.concatenate((cols, categoricals[i] + '_' + np.sort(cat_features.iloc[:, i].unique())))    
cat = pd.DataFrame(res, columns=cols)
# Total 252 categorical features
cat.shape # (1460, 252)
# Dropping original categorical variables
trainX = trainX.drop(categoricals, axis=1)
# Concatenating the One Hot Encoded variables to the train dataset
trainX = pd.concat([trainX, cat], axis=1)
# New data shape
trainX.shape # (1460, 289)
# Filling the NaN with median
trainX.fillna(trainX.median(), inplace=True)
# Normalizing training features
scalar = MinMaxScaler()
norm_train = pd.DataFrame(scalar.fit_transform(trainX), columns=trainX.columns)
# Normalizing training target
scalar_target = MinMaxScaler()
trainY = scalar_target.fit_transform(trainY.values.reshape(-1, 1))
# Defining the network
model = Sequential([
  Dense(norm_train.shape[1], input_dim=norm_train.shape[1], activation='sigmoid'),    
  Dense(units=norm_train.shape[1]//2, activation='sigmoid'),    
  Dense(units=1, activation='softmax'),
])


/home/wolf/anaconda3/envs/deepl-env/lib/python3.11/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
# Printing model summary
model.summary()
# _________________________________________________________________
# Layer (type)                 Output Shape              Param #   
# =================================================================
# dense_34 (Dense)             (None, 289)               83810     
# _________________________________________________________________
# dense_35 (Dense)             (None, 144)               41760     
# _________________________________________________________________
# dense_36 (Dense)             (None, 1)                 145       
# =================================================================
# Total params: 125,715
# Trainable params: 125,715
# Non-trainable params: 0


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 288)            │        83,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 144)            │        41,616 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           145 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 124,993 (488.25 KB)

 Trainable params: 124,993 (488.25 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# Compiling and Training Network
model.compile(optimizer='sgd', loss='mean_squared_error')
model.fit(trainX, trainY, batch_size=512, epochs=20, verbose=1, validation_split=0.2)


Epoch 1/20


/home/wolf/anaconda3/envs/deepl-env/lib/python3.11/site-packages/keras/src/ops/nn.py:944: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 175ms/step - loss: 0.6489 - val_loss: 0.6456
Epoch 2/20
1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.6555

/home/wolf/anaconda3/envs/deepl-env/lib/python3.11/site-packages/keras/src/ops/nn.py:944: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - loss: 0.6499 - val_loss: 0.6456
Epoch 3/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 0.6481 - val_loss: 0.6456
Epoch 4/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 0.6484 - val_loss: 0.6456
Epoch 5/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.6504 - val_loss: 0.6456
Epoch 6/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.6475 - val_loss: 0.6456
Epoch 7/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.6498 - val_loss: 0.6456
Epoch 8/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.6482 - val_loss: 0.6456
Epoch 9/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 0.6468 - val_loss: 0.6456
Epoch 10/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 0.6484 - val_loss: 0.6456
Epoch 11/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 0.6477 - val_loss: 0.6456
Epoch 12/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 0.6488 - val_loss: 0.6456
Epoch 13/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.6480 - val_loss: 0.6456
Epoch 14/20